In [146]:
import itertools
import numpy as np

In [263]:
#settings for 2x2 warehouse
#container_positions = [0,1,2,3] #amount of containers/ if storage needs 6 spaces list will be [0,1,2,3,4,5]
#travel_costs=[1,2,2,3] #defines the steps taken to reach container/ has to be modified when 6 storagespaces needed
#reward_distribution = [4,3,3,1] #reward reached when storing or restoring a container
#trainingsfile_name_in_root_dir = "SAKI_Exercise_3_warehousetraining2x2.txt"
#warehousefile_name_in_root_dir = "SAKI_Exercise_3_warehouseorder2x2.txt"

#settings for 3x2 warehouse
container_positions = [0,1,2,3,4,5] #amount of containers/ if storage needs 6 spaces list will be [0,1,2,3,4,5]
travel_costs=[1,2,2,3,3,4] #defines the steps taken to reach container/ has to be modified when 6 storagespaces needed
reward_distribution = [4,3,3,2,2,1] #reward reached when storing or restoring a container
trainingsfile_name_in_root_dir = "Exercise3-ReinforcementLearning-warehousetraining.txt"
warehousefile_name_in_root_dir = "Exercise3-ReinforcementLearning-warehouseorder.txt"

In [264]:
def gen_action_states(actionless_state, actions):
    tmp_action_states = []
    for action in actions:
        tmp_action_state = []
        for state in actionless_state:
            tmp_action_state.append(state)
        tmp_action_state.append(action)
        tmp_action_states.append(tuple(tmp_action_state))
    return tmp_action_states

def generate_states():
    storage_state = ["x","r","b","w"] #0 = empty, r = red, b = blue, w = white
    actionless_states = [p for p in itertools.product(storage_state, repeat=len(container_positions))]
    action_states = []
    actions = ["sr", "sb", "sw", "rr", "rb", "rw"]
    for state_without_action in actionless_states:
        action_states.extend(gen_action_states(state_without_action, actions))
    return action_states

In [265]:
import pandas as pd
def load_training_df():
    data = pd.read_csv(trainingsfile_name_in_root_dir, delimiter='\t', names=["action", "color"])
    data.head()
    df = data.copy()
    return df

def load_test_df():
    data = pd.read_csv(warehousefile_name_in_root_dir, delimiter='\t', names=["action", "color"])
    data.head()
    df = data.copy()
    return df

In [266]:
def calc_container_distribution(dataframe):
    count_of_trainingset = len(dataframe)
    distribution_df = dataframe.copy()
    print(count_of_trainingset)
    distribution_df = distribution_df.groupby(['action', 'color']).size().reset_index(name='count')
    distribution_df['count'] = distribution_df['count'].div(count_of_trainingset)
    return distribution_df

In [267]:
def get_color(state):
    return state[len(container_positions)][1].lower()

def get_actioncolor(state):
    return state[len(container_positions)].lower()

def has_warehouse_free_slots(state):
    for s in state:
        if s == "x":
            return True
    return False

def does_warehouse_store_color(state, color_to_restore):
    for s in state:
        if s == color_to_restore:
            return True
    return False


def is_warehouse_empty(state):
    for s in state:
        if s != 'x':
            return False
    return True

def distribute_transition_probability_matrix(tpm):
    for index, row_vector in enumerate(tpm):
        #print(index, row_vector)
        sum = np.sum(row_vector)
        if sum == 0:
            #print('error 0')
            # no transition was possible for this state (e. g. restore from empty warehouse)
            # it should stay in the state then since the sum of each row has to equal one
            tpm[index, index] = 1
            continue

        #for j, obj in enumerate(row_vector):
            #print(j, obj)
    # give every possible transition the same possibility by dividing every element by the sum
    # of the row. E. g. [0, 0, 1, 1, 0, 0] will be converted to [0, 0, 0.5, 0.5, 0, 0].
    # This will also result in the sum of the row equal to one.
    tpm = tpm / tpm.sum(axis=1)[:, None]
    return tpm

def is_store_action_possible(pos, state_1, state_2):
    if not has_warehouse_free_slots(state_1):
        return False

    color_to_store = get_color(state_1)

    for i in range(len(state_1)):
        if i == len(container_positions):
            break
        if i == pos:
            continue
        if state_1[i] != state_2[i]:
            return False
            
    if state_1[pos] == "x" and state_2[pos] != color_to_store and state_2[pos] != "x":
        # an empty slot was overwritten, but with something else we expected, not possible
        return False

    if state_1[pos] == "x" and state_2[pos] == color_to_store:
        return True
    
    return False


def is_restore_action_possible(pos, state_1, state_2):

    color_to_restore = get_color(state_1)
    if does_warehouse_store_color(state_1, color_to_restore) == False or is_warehouse_empty(state_1):
        # Warehouse does not store color, so we obviously can't restore it
        return False

    for i in range(len(state_1)):
        if i == len(container_positions):
            break
            
        if i == pos:
            continue
            
        if state_1[i] != state_2[i]:
            return False
            
    if state_1[pos] == "x":
        # an empty slot was overwritten, but with something else we expected, not possible
        return False
    
    if state_1[pos] == color_to_restore and state_2[pos] != "x" and state_2[pos] != color_to_restore:
        # an empty slot was overwritten, but with something else we expected, not possible
        return False

    if state_1[pos] == color_to_restore and state_2[pos] == 'x':
        return True
    
    return False

def is_transition_possible(pos, curr_state, next_state):
    #print("len of container_pos: ", len(container_positions))
    #print("curr_state: ",curr_state[len(container_positions)])
    curr_action = curr_state[len(container_positions)]
    if curr_action == "sb" or curr_action == "sr" or curr_action == "sw":
        return is_store_action_possible(pos, curr_state, next_state)
    else:
        return is_restore_action_possible(pos, curr_state, next_state)
    
def get_transition_probability_matrix(states):
    tp_matrix = np.zeros((len(states), len(states)), dtype=float, order='C')
    for x, curr_state in enumerate(states, start=0):
        for y, next_state in enumerate(states, start=0):
            if is_transition_possible(curr_state, next_state):
                tp_matrix[x, y] = 1
    return distribute_transition_probability_matrix(tp_matrix)


def create_tpm(states, distributions):
    tpm_all = []
    for pos in container_positions:
        tpm = np.zeros((len(states), len(states)),dtype=np.float16)
        print("tpm_shape: ",tpm.shape)
        for x, curr_state in enumerate(states, start=0):
            for y, next_state in enumerate(states, start=0):
                if is_transition_possible(pos, curr_state, next_state):
                    state_col = get_actioncolor(next_state)
                    if state_col == 'sr':
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'store') & (distribution_df['color'] == 'red'), 'count'].item()
                    elif state_col == 'sw':
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'store') & (distribution_df['color'] == 'white'), 'count'].item()
                    elif state_col == 'sb':
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'store') & (distribution_df['color'] == 'blue'), 'count'].item()
                    elif state_col == 'rr':
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'restore') & (distribution_df['color'] == 'red'), 'count'].item()
                    elif state_col == 'rw':
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'restore') & (distribution_df['color'] == 'white'), 'count'].item()
                    else:
                        tpm[x, y] = distribution_df.loc[(distribution_df['action'] == 'restore') & (distribution_df['color'] == 'blue'), 'count'].item()
                    # switch case for all coloractions possible and then add percentage
                   
                  
        distributed_tpm = distribute_transition_probability_matrix(tpm)
        tpm_all.append(distributed_tpm)
    
    return tpm_all

In [268]:

def get_store_reward(state, pos, reward_distribution):
    #print(state, state[pos], pos)
    if state[pos] != 'x':
        #print('full', state[pos], state, pos, reward_distribution[pos])
        return -1
    else:
        #print(reward_distribution[pos])
        col = get_color(state)
        fak = 10
        if col == 'r':
            fak *= distribution_df.loc[(distribution_df['action'] == 'store') & (distribution_df['color'] == 'red'), 'count'].item()
        if col == 'w':
            fak *= distribution_df.loc[(distribution_df['action'] == 'store') & (distribution_df['color'] == 'white'), 'count'].item()
        if col == 'b':
            fak *= distribution_df.loc[(distribution_df['action'] == 'store') & (distribution_df['color'] == 'blue'), 'count'].item()
        return reward_distribution[pos]*fak
    
def get_restore_reward(state, pos, reward_distribution):
    if state[pos] != get_color(state):
        return -1
    else:
        return reward_distribution[pos]

def get_reward_matrix(states, container_positions, reward_distribution):
    #action_rm = np.zeros((1,len(states)))
    rm = np.zeros((len(states), len(container_positions)))
    print(rm)
    for pos in container_positions:
        #print(i, pos)
        for j, state in enumerate(states, start=0):
            #print(state, state[len(positions)], state[len(positions)][0])
            if state[len(container_positions)][0] == 's':
                rm[j, pos] = get_store_reward(state, pos, reward_distribution)
            elif state[len(container_positions)][0] == 'r':
                rm[j, pos] = get_restore_reward(state, pos, reward_distribution)
            else:
                return False
                
    print(rm)
    return rm
    #%%
def determine_new_warehouse(warehouse, pos, action, color):
    col_symb = determine_col_symbol(color)
    #print("______________________________________________________________")
    #print("Before: ", warehouse, warehouse[pos], pos, action, color)
    if action == 'store' and warehouse[pos] == 'x':
        warehouse[pos] = col_symb
    elif action == 'restore' and warehouse[pos] != 'x':
        warehouse[pos] = 'x'
    #print("AFTER: ", warehouse, warehouse[pos], pos, action, color)
    #print("--------------------------------------------------------------")
    return warehouse

        
def determine_col_symbol(color):
    col_symb = ""
    if color == 'red':
        col_symb='r'
    if color == 'white':
        col_symb='w'
    if color == 'blue':
        col_symb='b'
    return col_symb

def get_warehouse_pos(policy, state):
    return policy[state]
    
def get_state_nr(warehouse):
    for index, state in enumerate(states, start=0):
        #print("--------------warehouse:---------------")
        #print(tuple(warehouse))
        #print("------<<--------state:---------<<------")
        #print(state)
        if state == tuple(warehouse):
            return index
    return 'failed'

def get_warehouse_state(warehouse, action, color):
    if action == 'store':
        if color == 'red':
            action='sr'
        if color == 'white':
            action='sw'
        if color == 'blue':
            action='sb'
    elif action == 'restore':
        if color == 'red':
            action='rr'
        if color == 'white':
            action='rw'
        if color == 'blue':
            action='rb'
    if len(warehouse) == len(container_positions):
        warehouse.append(action)
    elif len(warehouse) == len(container_positions)+1:
        warehouse[len(container_positions)] = action
    return warehouse
        
def test_algorithm(policy: tuple):
    warehouse = ["x"]*len(container_positions)
    #print(warehouse)
    #print(type(warehouse))
    test_df = load_test_df()
    field_to_go = 0
    warehouse_states = []
    for index, row in test_df.iterrows():
        action_warehouse = get_warehouse_state(warehouse, row.action, row.color)
        state = get_state_nr(action_warehouse)
        #print(action_warehouse, state)
        if state != 'failed':
            #print('test')
            warehouse_position = get_warehouse_pos(policy, state)
            #print("warehouse_pos: ", warehouse_position)
            field_to_go += travel_costs[warehouse_position]*2
            #print("field to go: ", field_to_go)
            warehouse = determine_new_warehouse(warehouse, warehouse_position, row.action, row.color)
            warehouse_tuple = tuple(warehouse)
            warehouse_states.append(warehouse_tuple)
            #print(warehouse)
        else:
            print(action_warehouse, state)
            return 'failed'
    result = [field_to_go, warehouse_states]
    return result

def store_in_next_empty(warehouse, color, travel_costs):
    col_symb = determine_col_symbol(color)
    for index, pos in enumerate(warehouse, start=0):
        if pos == 'x':
            warehouse[index] = col_symb
            result = [travel_costs[index]*2, warehouse]
            return result
        
    return False

def get_correct_first_container(warehouse, color, travel_costs):
    col_symb = determine_col_symbol(color)
    for index, pos in enumerate(warehouse, start=0):
        if pos == col_symb:
            warehouse[index] = "x"
            result = [travel_costs[index]*2, warehouse]
            return result
        
    return False

def greedy_distance(travel_costs):
    warehouse = ["x"]*len(container_positions)
    test_df = load_test_df()
    distance_traveled = 0
    warehouse_states = []
    for index, row in test_df.iterrows():
        warehouse_state = ""
        if row.action == 'store':
            warehouse_state = store_in_next_empty(warehouse, row.color, travel_costs)
            warehouse_tuple_store = tuple(warehouse_state[1])
            warehouse_states.append(warehouse_tuple_store)
            distance_traveled += warehouse_state[0]
            #print(distance_traveled)
        elif row.action == 'restore':
            warehouse_state = get_correct_first_container(warehouse, row.color, travel_costs)
            warehouse_tuple_restore = tuple(warehouse_state[1])
            warehouse_states.append(warehouse_tuple_restore)
            distance_traveled += warehouse_state[0]
            #print(distance_traveled)
    
    print(warehouse_states)        
    result = [distance_traveled, warehouse_states]
    return result

In [269]:
#Generate states and transition probability matrix
states = generate_states()
trainingsfile_df = load_training_df()
distribution_df = calc_container_distribution(trainingsfile_df)
test1 = distribution_df.loc[(distribution_df['action'] == 'store') & (distribution_df['color'] == 'red'), 'count'].item()
print(test1)
tpm = create_tpm(states, distribution_df)
print(distribution_df)
#print(tpm)

12108
0.24686157912124215
tpm_shape:  (24576, 24576)
tpm_shape:  (24576, 24576)
tpm_shape:  (24576, 24576)
tpm_shape:  (24576, 24576)
tpm_shape:  (24576, 24576)
tpm_shape:  (24576, 24576)
    action  color     count
0  restore   blue  0.125289
1  restore    red  0.246862
2  restore  white  0.127849
3    store   blue  0.125289
4    store    red  0.246862
5    store  white  0.127849


In [270]:
#generate reward matrix
reward_matrix = get_reward_matrix(states, container_positions, reward_distribution)


[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[ 9.87446316  7.40584737  7.40584737  4.93723158  4.93723158  2.46861579]
 [ 5.0115626   3.75867195  3.75867195  2.5057813   2.5057813   1.25289065]
 [ 5.11397423  3.83548067  3.83548067  2.55698712  2.55698712  1.27849356]
 ...
 [-1.         -1.         -1.         -1.         -1.         -1.        ]
 [-1.         -1.         -1.         -1.         -1.         -1.        ]
 [ 4.          3.          3.          2.          2.          1.        ]]


In [273]:
#run algorithms
import mdptoolbox

mdpResultPolicy = mdptoolbox.mdp.PolicyIteration(tpm, reward_matrix, 0.2, max_iter=20)
mdpResultValue = mdptoolbox.mdp.ValueIteration(tpm, reward_matrix, 0.95, max_iter=len(states) * 5)
# Run the MDP
mdpResultPolicy.run()
mdpResultValue.run()

print('PolicyIteration:')
print(mdpResultPolicy.policy)
print(mdpResultPolicy.V)
print(mdpResultPolicy.iter)

print('ValueIteration:')
print(mdpResultValue.policy)
print(mdpResultValue.V)
print(mdpResultValue.iter)



PolicyIteration:
(0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 5, 0, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5, 4, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 5, 0, 0, 0, 0, 0, 4, 0, 0, 0, 5, 0, 4, 0, 0, 0, 0, 5, 4, 0, 0, 0, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 5, 0, 0, 0, 0, 3, 0, 5, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 5, 0, 0, 0, 0, 3, 0, 5, 0, 0, 0, 3, 4, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 3, 4, 5, 0, 0, 0, 3, 0, 4, 0, 0, 0, 3, 0, 4, 0, 0, 0, 3, 5, 4, 0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3, 0, 0, 0, 0, 5, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 5, 0, 0, 0, 4, 3, 0, 0, 0, 0, 4, 3, 0, 0, 0, 0, 4, 3, 0, 0, 0, 0, 4, 3, 5, 0, 0, 0, 0, 3, 0, 0, 0, 0, 5, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 5, 3, 4, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 3, 0, 0, 0, 5, 0, 3, 0, 0, 0, 0, 5, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 4, 0, 3, 0, 0, 0, 4, 0, 3, 0, 0, 0, 4

In [274]:
#test algorithms with dataset
traveled_fields_mdpResultValue = test_algorithm(mdpResultValue.policy)
traveled_fields_mdpResultPolicy = test_algorithm(mdpResultPolicy.policy)
traveled_fields_greedy = greedy_distance(travel_costs)
print("ValueIteration Robot traveled: ", traveled_fields_mdpResultValue[0])
print("PolicyIteration Robot traveled: ", traveled_fields_mdpResultPolicy[0])
print("Greedy Robot traveled: ", traveled_fields_greedy[0])
value_iter_states = traveled_fields_mdpResultValue[1]
policy_iter_states = traveled_fields_mdpResultPolicy[1]
greedy_states = traveled_fields_greedy

[('r', 'x', 'x', 'x', 'x', 'x'), ('r', 'b', 'x', 'x', 'x', 'x'), ('r', 'b', 'w', 'x', 'x', 'x'), ('r', 'x', 'w', 'x', 'x', 'x'), ('r', 'x', 'x', 'x', 'x', 'x'), ('r', 'w', 'x', 'x', 'x', 'x'), ('r', 'w', 'r', 'x', 'x', 'x'), ('r', 'w', 'r', 'w', 'x', 'x'), ('r', 'w', 'r', 'w', 'r', 'x'), ('x', 'w', 'r', 'w', 'r', 'x'), ('x', 'w', 'x', 'w', 'r', 'x'), ('w', 'w', 'x', 'w', 'r', 'x'), ('w', 'w', 'b', 'w', 'r', 'x'), ('w', 'w', 'b', 'w', 'r', 'w'), ('w', 'w', 'b', 'w', 'x', 'w'), ('x', 'w', 'b', 'w', 'x', 'w'), ('x', 'x', 'b', 'w', 'x', 'w'), ('r', 'x', 'b', 'w', 'x', 'w'), ('r', 'b', 'b', 'w', 'x', 'w'), ('x', 'b', 'b', 'w', 'x', 'w'), ('b', 'b', 'b', 'w', 'x', 'w'), ('b', 'b', 'b', 'w', 'w', 'w'), ('b', 'b', 'b', 'x', 'w', 'w'), ('x', 'b', 'b', 'x', 'w', 'w'), ('x', 'b', 'b', 'x', 'x', 'w'), ('x', 'b', 'b', 'x', 'x', 'x'), ('x', 'x', 'b', 'x', 'x', 'x'), ('x', 'x', 'x', 'x', 'x', 'x'), ('r', 'x', 'x', 'x', 'x', 'x'), ('x', 'x', 'x', 'x', 'x', 'x'), ('r', 'x', 'x', 'x', 'x', 'x'), ('r', '